Solving the Traveling Salesman Problem with QAOA

In [4]:
import numpy as np
%matplotlib inline

def points_order_to_binary_state(points_order):
    """
    Transforms the order of points from the standard representation: [0, 1, 2],
    to the binary one: [1,0,0,0,1,0,0,0,1]
    """
    number_of_points = len(points_order)
    binary_state = np.zeros((len(points_order))**2)
    for j in range(len(points_order)):
        p = points_order[j]
        binary_state[(number_of_points) * (j) + (p)] = 1
    return binary_state

def binary_state_to_points_order(binary_state):
    """
    Transforms the the order of points from the binary representation: [1,0,0,0,1,0,0,0,1],
    to the binary one: [0, 1, 2]
    """
    points_order = []
    number_of_points = int(np.sqrt(len(binary_state)))
    for p in range(number_of_points):
        for j in range(number_of_points):
            if binary_state[(number_of_points) * p + j] == 1:
                points_order.append(j)
    return points_order

In [6]:
import utilities
cities = np.array([[0, 4],[0, 0],[3, 0]])
distance_matrix = utilities.get_distance_matrix(cities)
print(distance_matrix)
number_of_cities = len(cities)

[[0. 4. 5.]
 [4. 0. 3.]
 [5. 3. 0.]]


In [7]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter

def create_qc_mix(nqubits, beta):
    nqubits = len(cities)**2
    qc_mix = QuantumCircuit(nqubits)
    for i in range(0, nqubits):
        qc_mix.rx(-1 * 2 * beta, i)
    return qc_mix

def create_qc_0(nqubits):
    qc_0 = QuantumCircuit(nqubits)
    for i in range(0, nqubits):
        qc_0.h(i)
    return qc_0

def create_qc_p(nqubits, gamma):
    number_of_cities = len(cities)
    qc_p = QuantumCircuit(nqubits)
    for i in range(number_of_cities):
        for j in range(i, number_of_cities):
            for t in range(number_of_cities - 1):
                if distance_matrix[i][j] != 0:
                    weight = distance_matrix[i][j]/2
                    qubit_1 = t * number_of_cities + i
                    qubit_2 = (t + 1) * number_of_cities + j
                    qc_p.rzz(2*gamma*weight, qubit_1, qubit_2)
    return qc_p

In [8]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info.operators import Operator
from scipy import linalg

def create_qaoa_circuit(nqubits, theta):
    qc_qaoa = QuantumCircuit(nqubits)
    beta = theta[0]
    gamma = theta[1]
    
    nqubits = len(cities)**2
    for i in range(0, nqubits):
        qc_qaoa.rx(-1 * 2 * beta, i)

    for i in range(0, nqubits):
        qc_qaoa.h(i)

    number_of_cities = len(cities)
    for i in range(number_of_cities):
        for j in range(i, number_of_cities):
            for t in range(number_of_cities - 1):
                if distance_matrix[i][j] != 0:
                    weight = distance_matrix[i][j]/2
                    qubit_1 = t * number_of_cities + i
                    qubit_2 = (t + 1) * number_of_cities + j
                    qc_qaoa.rzz(2*gamma*weight, qubit_1, qubit_2)
    
    weight = -100 * np.max(distance_matrix)
    weight_op2 = weight/8.0

    #bilocation
    number_of_nodes = len(distance_matrix[0])
    for t in range(number_of_nodes):
        range_of_qubits = list(range(t * number_of_nodes, (t + 1) * number_of_nodes))
        op = Operator(linalg.expm(-(1j)*2*gamma*(np.diag([1,0,0,1,0,1,1,1])*weight)))
        qc_qaoa.append(op, range_of_qubits)
        
    #repetition
    for i in range(number_of_nodes):
        range_of_qubits = list(range(i, number_of_nodes**2, number_of_nodes))
        op = Operator(linalg.expm(-(1j)*2*gamma*(np.diag([1,0,0,1,0,1,1,1])*weight)))
        qc_qaoa.append(op, range_of_qubits)

    qc_qaoa.measure_all()
    
    return qc_qaoa

In [9]:
def compute_expectation(counts, distance_matrix):
    avg = 0
    sum_count = 0
    counts_list = [(k, v) for k, v in counts.items()]
    counts_list = [(binary_state_to_points_order([int(i) for i in x[0]]), x[1]) for x in counts_list]
    for item in counts_list:
        bitstring = item[0]
        count = item[1]
        path = binary_state_to_points_order(bitstring)
        total = 0
        for i in range(len(path) - 1):
            total += distance_matrix[path[i]][path[i+1]]
        avg += total * int(count)
        sum_count += int(count)
    return avg/sum_count

def get_expectation(distance_matrix, shots=512):
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        qc = create_qaoa_circuit(len(distance_matrix[0])**2, theta)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=512).result().get_counts()
        
        return compute_expectation(counts, distance_matrix)
    
    return execute_circ


In [10]:
from scipy.optimize import minimize

expectation = get_expectation(distance_matrix)

res = minimize(expectation, 
                      [1.0, 1.0], 
                      method='COBYLA')
res

     fun: 0.58984375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 13
  status: 1
 success: True
       x: array([1., 1.])

In [11]:
from qiskit.visualization import plot_histogram

backend = Aer.get_backend('aer_simulator')
backend.shots = 512

qc_res = create_qaoa_circuit(len(distance_matrix[0])**2, res.x)
counts = backend.run(qc_res, seed_simulator=10).result().get_counts()
counts_list = [(k, v) for k, v in counts.items()]
counts_list.sort(key = lambda x: x[1])
counts_list = [(binary_state_to_points_order([int(i) for i in x[0]]), x[1]) for x in counts_list]
print(counts_list)
#plot_histogram(counts)

[([1, 2], 1), ([0, 0, 2], 1), ([1, 2, 0, 1, 2], 1), ([1, 2, 1, 2, 1], 1), ([0, 2, 1], 1), ([1, 2, 1, 0, 1, 2], 1), ([0, 2, 0, 1, 0], 1), ([0, 2, 0, 1, 0, 1], 1), ([1, 0, 2, 0, 2], 1), ([0, 2, 0], 1), ([2, 2], 1), ([0, 1, 2, 0], 1), ([0, 1, 1], 1), ([0, 1, 0, 2, 2], 1), ([0, 1, 2, 1], 1), ([0, 0, 0], 1), ([0, 2, 1, 1, 2], 1), ([1, 2, 0, 1], 1), ([0, 0, 2, 0], 1), ([2, 0], 1), ([1, 2, 0], 1), ([0, 2, 2], 1), ([1, 0, 2, 0], 1), ([2, 1, 2, 1, 2], 1), ([1, 1, 2], 1), ([0, 2, 0, 1, 2, 0, 1], 1), ([1, 2, 1, 2], 1), ([0, 2, 0, 2], 1), ([0, 1, 2, 0, 1], 1), ([0, 1, 2, 2], 1), ([1, 1, 0, 2], 1), ([0, 0, 1], 1), ([1, 1, 2, 0, 1], 1), ([0, 0], 1), ([0, 1, 2], 1), ([0], 1), ([0, 1, 2, 0, 2], 1), ([0, 1, 0, 1, 2], 1), ([2, 0, 1, 2, 0], 1), ([1, 2, 2, 1], 1), ([0, 1, 1, 0, 1, 2], 1), ([1, 2, 0, 2, 1, 2], 1), ([1, 2, 0, 2, 0], 1), ([0, 1, 2, 0], 1), ([2, 0, 2, 1], 1), ([2, 1], 1), ([1, 2, 0, 1], 1), ([0, 1, 2], 1), ([0, 1, 2, 1, 2], 1), ([1, 2, 1, 1, 2], 1), ([1, 1, 2, 1], 1), ([1, 2, 0, 2, 0, 2], 1),